In [1]:
%load_ext autoreload
%autoreload 2

import nest
import yaml
from pathlib import Path
from pprint import pprint

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from denest import *

In [3]:
PARAMS_DIR = Path('./params')
OUTPUT_DIR = Path('./output')

# Let's familiarize ourselves with the ``Network`` object

__...by building the elements of the network one by one__

In this section we:

1. Initialize an empty ``Network`` object
2. __Initialize all the elements of the network__ using  the ``Network.build_*`` methods
    1. Models (neuron models, recorder models, synapse models
    2. Layers
    3. Connection models
    4. Individual connections
    5. Population and connection recorders
3. __Create the network__ in NEST
4. __Access the network elements__ (GIDs, etc)
4. Export and reuse the parameter tree allowing us to __replicate the network__

## Initialize an empty network

When initialized without argument or with an empty tree as an argument, all the expected subtrees are initialized as empty.

When building the elements interactively, the network's parameters are updated

In [4]:
net = Network()

2020-05-29 08:39:53,969 [denest.utils.validation] INFO: 'None' tree: adding empty child neuron_models
2020-05-29 08:39:53,970 [denest.utils.validation] INFO: 'None' tree: adding empty child synapse_models
2020-05-29 08:39:53,971 [denest.utils.validation] INFO: 'None' tree: adding empty child layers
2020-05-29 08:39:53,972 [denest.utils.validation] INFO: 'None' tree: adding empty child connection_models
2020-05-29 08:39:53,973 [denest.utils.validation] INFO: 'None' tree: adding empty child topology
2020-05-29 08:39:53,974 [denest.utils.validation] INFO: 'None' tree: adding empty child recorder_models
2020-05-29 08:39:53,975 [denest.utils.validation] INFO: 'None' tree: adding empty child recorders
2020-05-29 08:39:53,978 [denest.network] INFO: Build N=0 ``Model`` objects
2020-05-29 08:39:53,980 [denest.network] INFO: Build N=0 ``SynapseModel`` objects
2020-05-29 08:39:53,984 [denest.network] INFO: Build N=0 ``Model`` objects
2020-05-29 08:39:53,987 [denest.network] INFO: Build N=0 ``Laye

In [5]:
# Network parameter tree is empty:
net.tree

ParamsTree(name='None', parent=None)
  params: {}
  nest_params: {}
  neuron_models:
    params: {}
    nest_params: {}
  synapse_models:
    params: {}
    nest_params: {}
  layers:
    params: {}
    nest_params: {}
  connection_models:
    params: {}
    nest_params: {}
  topology:
    params: {}
    nest_params: {}
  recorder_models:
    params: {}
    nest_params: {}
  recorders:
    params: {}
    nest_params: {}
  

## Build the network components (models, layers, connections, recorders)


### Define new models

We can define neuron, recorder, stimulator and synapse models with arbitrary parameters from parameter trees.

Each leaf corresponds to a new (named) model. Its ``nest_params`` and ``params`` are hierarchically inherited.

The ``nest_model`` used is specified in the leaf's ``params``

#### ``'neuron_models'`` tree:

Initalize ``Network.neuron_models`` with the ``Network.build_neuron_models`` method

In [6]:
neuron_models_tree = ParamsTree.read(PARAMS_DIR/'models.yml').children['neuron_models']
pprint(neuron_models_tree)

ParamsTree(name='neuron_models', parent='None')
  params: {}
  nest_params: {}
  my_neuron:
    params:
      nest_model: ht_neuron
    nest_params:
      g_KL: 1.0
      g_NaL: 1.0
    l1_exc:
      params: {}
      nest_params:
        V_m: -44.0
    l1_inh:
      params: {}
      nest_params:
        V_m: -55.0
  


In [7]:
net.build_neuron_models(neuron_models_tree)

2020-05-29 08:39:54,182 [denest.network] INFO: Build N=2 ``Model`` objects


In [8]:
# The neuron models are saved as an attribute for the Network object
print("\n``Network.neuron_models`` :")
pprint(net.neuron_models)


``Network.neuron_models`` :
{'l1_exc': Model(l1_exc, {'nest_model': 'ht_neuron'}, {'V_m': -44.0, 'g_KL': 1.0, 'g_NaL': 1.0}),
 'l1_inh': Model(l1_inh, {'nest_model': 'ht_neuron'}, {'V_m': -55.0, 'g_KL': 1.0, 'g_NaL': 1.0})}


In [9]:
pprint(net.neuron_models['l1_exc'])
pprint(net.neuron_models['l1_exc'].params)
pprint(net.neuron_models['l1_exc'].nest_params)

Model(l1_exc, {'nest_model': 'ht_neuron'}, {'V_m': -44.0, 'g_KL': 1.0, 'g_NaL': 1.0})
{'nest_model': 'ht_neuron'}
{'V_m': -44.0, 'g_KL': 1.0, 'g_NaL': 1.0}


#### ``'recorder_models'`` tree:

Initalize ``Network.recorder_models`` with the ``Network.build_recorder_models`` method

-> Same thing as for neuron models

In [10]:
# ``Network.build_*`` methods accept as argument ``ParamsTree`` objects, but also tree-like dictionaries 
recorder_models_tree = ParamsTree.read(PARAMS_DIR/'models.yml').children['recorder_models']
recorder_models_tree

ParamsTree(name='recorder_models', parent='None')
  params: {}
  nest_params:
    record_to:
    - memory
    - file
  weight_recorder:
    params:
      nest_model: weight_recorder
    nest_params: {}
  my_multimeter:
    params:
      nest_model: multimeter
    nest_params:
      record_from:
      - V_m
  

In [11]:
# ``Network.build_*`` methods accept as argument ``ParamsTree`` objects, but also tree-like dictionaries 
recorder_models_tree = recorder_models_tree.asdict()
recorder_models_tree

{'params': {},
 'nest_params': {'record_to': ['memory', 'file']},
 'weight_recorder': {'params': {'nest_model': 'weight_recorder'},
  'nest_params': {}},
 'my_multimeter': {'params': {'nest_model': 'multimeter'},
  'nest_params': {'record_from': ['V_m']}}}

In [12]:
net.build_recorder_models(recorder_models_tree)

2020-05-29 08:39:54,522 [denest.network] INFO: Build N=2 ``Model`` objects


In [13]:
print("\n``Network.recorder_models`` :")
pprint(net.recorder_models)


``Network.recorder_models`` :
{'my_multimeter': Model(my_multimeter, {'nest_model': 'multimeter'}, {'record_from': ['V_m'], 'record_to': ['memory', 'file']}),
 'weight_recorder': Model(weight_recorder, {'nest_model': 'weight_recorder'}, {'record_to': ['memory', 'file']})}


#### ``'synapse_model'`` tree:

Initalize ``Network.synapse_models`` with the ``Network.build_synapse_model`` method

-> Same thing as for neuron models, with as a bonus a convenient way of specifying the receptor type of the synapse

-> If specifying the ``receptor_type`` and ``target_model`` in the ``SynapseModel`` params, the corresponding port is determined automatically

In [14]:
synapse_models_tree = ParamsTree.read(PARAMS_DIR/'models.yml').children['synapse_models']
synapse_models_tree

ParamsTree(name='synapse_models', parent='None')
  params: {}
  nest_params: {}
  my_AMPA_synapse:
    params:
      nest_model: ht_synapse
      receptor_type: AMPA
      target_neuron: ht_neuron
    nest_params: {}
  my_GABAA_synapse:
    params:
      nest_model: ht_synapse
      receptor_type: GABA_A
      target_neuron: ht_neuron
    nest_params: {}
  

In [15]:
net.build_synapse_models(synapse_models_tree)

2020-05-29 08:39:54,688 [denest.network] INFO: Build N=2 ``SynapseModel`` objects


In [16]:
print("\n``Network.synapse_models`` :")
pprint(net.synapse_models)


``Network.synapse_models`` :
{'my_AMPA_synapse': SynapseModel(my_AMPA_synapse, {'nest_model': 'ht_synapse'}, {'receptor_type': 1}),
 'my_GABAA_synapse': SynapseModel(my_GABAA_synapse, {'nest_model': 'ht_synapse'}, {'receptor_type': 3})}


Note that the ``receptor_type`` nest_parameter was inferred

### Define layers

As for models, we can create ``nest.Topology`` layers from the leaves of a tree.

The elements can be nest models with their default parameters, or the ones we just created with custom params

For layers of stimulator devices, we can use the ``InputLayer`` object, which can automatically create paired parrot neurons for each stimulator units, by adding ``type: 'InputLayer'``
to the params


#### ``'layers'`` tree:


In [17]:
layer_tree = ParamsTree.read(PARAMS_DIR/'layers.yml')
layer_tree

ParamsTree(name='None', parent=None)
  params: {}
  nest_params: {}
  layers:
    params:
      type: null
    nest_params:
      rows: 5
      columns: 5
      extent:
      - 5.0
      - 5.0
      edge_wrap: true
    input_layer:
      params:
        type: InputLayer
        populations:
          poisson_generator: 1
      nest_params: {}
    l1:
      params:
        populations:
          l1_exc: 4
          l1_inh: 2
      nest_params: {}
  

In [18]:
net.build_layers(layer_tree)

2020-05-29 08:39:54,851 [denest.network] INFO: Build N=2 ``Layer`` or ``InputLayer`` objects.


In [19]:
pprint(net.layers)

{'input_layer': InputLayer(input_layer, {'populations': {'parrot_neuron': 1, 'poisson_generator': 1},
 'type': 'InputLayer'}, {'columns': 5,
 'edge_wrap': True,
 'elements': ('poisson_generator', 1, 'parrot_neuron', 1),
 'extent': [5.0, 5.0],
 'rows': 5}),
 'l1': Layer(l1, {'populations': {'l1_exc': 4, 'l1_inh': 2}, 'type': None}, {'columns': 5,
 'edge_wrap': True,
 'elements': ('l1_exc', 4, 'l1_inh', 2),
 'extent': [5.0, 5.0],
 'rows': 5})}


In [20]:
print("'l1' layer")
pprint(net.layers['l1'].params)
pprint(net.layers['l1'].nest_params)

'l1' layer
{'populations': {'l1_exc': 4, 'l1_inh': 2}, 'type': None}
{'columns': 5,
 'edge_wrap': True,
 'elements': ('l1_exc', 4, 'l1_inh', 2),
 'extent': [5.0, 5.0],
 'rows': 5}


### Define connections

We create connections using a two step process:

1. Create ``ConnectionModel`` objects from a tree. Each named leaf will define a template from which individual connections can inherit their parameters
2. Create ``Connection`` objects from a list, specifying for each item the source layer x population, target layer x population and the connection model to inherit parameters from

#### 1- Define templates from the `connection_models` tree



In [21]:
conn_model_tree = ParamsTree.read(PARAMS_DIR/'connections.yml').children['connection_models']
conn_model_tree

ParamsTree(name='connection_models', parent='None')
  params: {}
  nest_params:
    connection_type: divergent
    mask:
      circular:
        radius: 2.0
    kernel: 1.0
  conn_1_AMPA:
    params: {}
    nest_params:
      synapse_model: my_AMPA_synapse
      weights: 1.0
  conn_2_GABAA:
    params: {}
    nest_params:
      synapse_model: my_GABAA_synapse
      weights: 2.0
  

In [22]:
net.build_connection_models(conn_model_tree)

2020-05-29 08:39:55,085 [denest.utils.validation] INFO: Object `conn_1_AMPA`: params: using default value for optional parameters:
{'type': 'topological'}
2020-05-29 08:39:55,088 [denest.utils.validation] INFO: Object `conn_2_GABAA`: params: using default value for optional parameters:
{'type': 'topological'}
2020-05-29 08:39:55,091 [denest.network] INFO: Build N=2 ``ConnectionModel`` objects


In [23]:
net.connection_models

{'conn_1_AMPA': ConnectionModel(conn_1_AMPA, 
  {'type': 'topological'}{'connection_type': 'divergent',
   'mask': {'circular': {'radius': 2.0}},
   'kernel': 1.0,
   'synapse_model': 'my_AMPA_synapse',
   'weights': 1.0}),
 'conn_2_GABAA': ConnectionModel(conn_2_GABAA, 
  {'type': 'topological'}{'connection_type': 'divergent',
   'mask': {'circular': {'radius': 2.0}},
   'kernel': 1.0,
   'synapse_model': 'my_GABAA_synapse',
   'weights': 2.0})}

### 2- Define individual connections from the `topology` tree

The list of connections is defined in the `connections` params of the `topology` tree

Check out the doc of `Network.build_connections` for expected formatting

In [24]:
conns_tree = ParamsTree.read(PARAMS_DIR/'connections.yml').children['topology']
conns_tree

ParamsTree(name='topology', parent='None')
  params:
    connections:
    - source_layers:
      - input_layer
      source_population: parrot_neuron
      target_layers:
      - l1
      target_population: l1_exc
      connection_model: conn_1_AMPA
    - source_layers:
      - l1
      source_population: l1_exc
      target_layers:
      - l1
      target_population: l1_inh
      connection_model: conn_1_AMPA
    - source_layers:
      - l1
      source_population: l1_inh
      target_layers:
      - l1
      target_population: l1_exc
      connection_model: conn_2_GABAA
  nest_params: {}
  

In [25]:
net.connection_models

{'conn_1_AMPA': ConnectionModel(conn_1_AMPA, 
  {'type': 'topological'}{'connection_type': 'divergent',
   'mask': {'circular': {'radius': 2.0}},
   'kernel': 1.0,
   'synapse_model': 'my_AMPA_synapse',
   'weights': 1.0}),
 'conn_2_GABAA': ConnectionModel(conn_2_GABAA, 
  {'type': 'topological'}{'connection_type': 'divergent',
   'mask': {'circular': {'radius': 2.0}},
   'kernel': 1.0,
   'synapse_model': 'my_GABAA_synapse',
   'weights': 2.0})}

In [26]:
net.build_connections(conns_tree)

2020-05-29 08:39:55,303 [denest.network] INFO: Build N=3 ``TopoConnection`` objects


In [27]:
net.connections

[TopoConnection(conn_1_AMPA-input_layer-parrot_neuron-l1-l1_exc, 
  {'type': 'topological'}{'connection_type': 'divergent',
   'mask': {'circular': {'radius': 2.0}},
   'kernel': 1.0,
   'synapse_model': 'my_AMPA_synapse',
   'weights': 1.0,
   'sources': {'model': 'parrot_neuron'},
   'targets': {'model': 'l1_exc'}}),
 TopoConnection(conn_1_AMPA-l1-l1_exc-l1-l1_inh, 
  {'type': 'topological'}{'connection_type': 'divergent',
   'mask': {'circular': {'radius': 2.0}},
   'kernel': 1.0,
   'synapse_model': 'my_AMPA_synapse',
   'weights': 1.0,
   'sources': {'model': 'l1_exc'},
   'targets': {'model': 'l1_inh'}}),
 TopoConnection(conn_2_GABAA-l1-l1_inh-l1-l1_exc, 
  {'type': 'topological'}{'connection_type': 'divergent',
   'mask': {'circular': {'radius': 2.0}},
   'kernel': 1.0,
   'synapse_model': 'my_GABAA_synapse',
   'weights': 2.0,
   'sources': {'model': 'l1_inh'},
   'targets': {'model': 'l1_exc'}})]

## Define recorders from the `recorders` tree

Similarly to the `topology` tree, recorders are defined from lists.


We separate recorders connected to synapses (eg weight recorder) and those connected to units (eg spike detectors),
which are defined in the `connection_recorders` and `population_recorders` params (resp.) of the `recorders` tree.


Check out the doc of the ``Network.build_recorders``, ``Network.build_population_recorders`` and
``Network.build_connection_recorders`` methods for expected formatting

The parameters of the recorders can be changed by using custom recorder models (in the `recorder_models` tree, see above) 

In [28]:
recorders_tree = ParamsTree.read(PARAMS_DIR/'recorders.yml').children['recorders']
recorders_tree

ParamsTree(name='recorders', parent='None')
  params:
    population_recorders:
    - layers:
      - l1
      populations:
      - l1_exc
      model: my_multimeter
    connection_recorders:
    - source_layers:
      - l1
      source_population: l1_exc
      target_layers:
      - l1
      target_population: l1_inh
      connection_model: conn_1_AMPA
      model: weight_recorder
  nest_params: {}
  

In [29]:
net.build_recorders(recorders_tree)

2020-05-29 08:39:55,471 [denest.network] INFO: Build N=1 population recorders.
2020-05-29 08:39:55,484 [denest.network] INFO: Build N=1 connection recorders.


In [30]:
net.population_recorders

[PopulationRecorder(my_multimeter_l1_l1_exc,  {}{})]

In [31]:
net.connection_recorders

[ConnectionRecorder(weight_recorder_conn_1_AMPA-l1-l1_exc-l1-l1_inh,  {}{})]

## Create the network

In [32]:
nest.ResetKernel()

In [33]:
net.create()

2020-05-29 08:39:55,686 [denest.network] INFO: Creating neuron models...
100%|██████████| 2/2 [00:00<00:00, 1480.52it/s]
2020-05-29 08:39:55,716 [denest.network] INFO: Creating synapse models...
100%|██████████| 2/2 [00:00<00:00, 951.41it/s]
2020-05-29 08:39:55,725 [denest.network] INFO: Creating recorder models...
100%|██████████| 2/2 [00:00<00:00, 1098.27it/s]
2020-05-29 08:39:55,732 [denest.network] INFO: Creating layers...
100%|██████████| 2/2 [00:00<00:00,  9.94it/s]
2020-05-29 08:39:55,941 [denest.network] INFO: Creating population recorders...
100%|██████████| 1/1 [00:00<00:00, 34.10it/s]
2020-05-29 08:39:55,976 [denest.network] INFO: Creating connection recorders...
100%|██████████| 1/1 [00:00<00:00, 86.22it/s]
2020-05-29 08:39:56,050 [denest.network] INFO: Connecting layers...
100%|██████████| 3/3 [00:00<00:00, 359.58it/s]
2020-05-29 08:39:56,134 [denest.network] INFO: Network size (including recorders and parrot neurons):
Number of nodes: 205
Number of connections: 6625


## Examine the network

denest provides convenient ways of accessing the objects in nest

### Check the defaults of the created models

In [34]:
print("`l1_exc` neuron models `nest_params`: ", net.neuron_models['l1_exc'].nest_params)

`l1_exc` neuron models `nest_params`:  {'g_KL': 1.0, 'g_NaL': 1.0, 'V_m': -44.0}


In [35]:
print('Corresponding params of the `l1_exc` model in nest:', nest.GetDefaults('l1_exc', list(net.neuron_models['l1_exc'].nest_params.keys())))

Corresponding params of the `l1_exc` model in nest: (1.0, 1.0, -44.0)


### Access the layers' units

In [36]:
net.layers['l1'].gids(location=(0, 0), population='l1_exc')

[53, 78, 103, 128]

### Access the connections created in NEST

In [37]:
conn = net.connections[0]
conn

TopoConnection(conn_1_AMPA-input_layer-parrot_neuron-l1-l1_exc, 
{'type': 'topological'}{'connection_type': 'divergent',
 'mask': {'circular': {'radius': 2.0}},
 'kernel': 1.0,
 'synapse_model': 'my_AMPA_synapse',
 'weights': 1.0,
 'sources': {'model': 'parrot_neuron'},
 'targets': {'model': 'l1_exc'}})

In [38]:
nest_conns = nest.GetConnections(
    source=conn.source.gids(conn.source_population),
    target=conn.target.gids(conn.target_population),
    synapse_model=conn.nest_params['synapse_model']
)
nest_conns[0:5]

(array('l', [27, 53, 0, 68, 0]),
 array('l', [27, 88, 0, 68, 1]),
 array('l', [27, 83, 0, 68, 2]),
 array('l', [27, 152, 0, 68, 3]),
 array('l', [27, 84, 0, 68, 4]))

### Access the recorders

In [39]:
rec = net.population_recorders[0]
print(rec, rec.gid, rec.model, rec.layer, rec.population_name)

my_multimeter_l1_l1_exc (203,) my_multimeter l1 l1_exc


In [40]:
connrec = net.connection_recorders[0]
print(connrec, connrec.gid, connrec.model)

weight_recorder_conn_1_AMPA-l1-l1_exc-l1-l1_inh (204,) weight_recorder


## Save and recreate the network

When building each of the network's elements using the `Network.build_*` methods, the `Network.tree` ParamsTree was updated

In [41]:
net.tree

ParamsTree(name='None', parent=None)
  params: {}
  nest_params: {}
  neuron_models:
    params: {}
    nest_params: {}
    my_neuron:
      params:
        nest_model: ht_neuron
      nest_params:
        g_KL: 1.0
        g_NaL: 1.0
      l1_exc:
        params: {}
        nest_params:
          V_m: -44.0
  
  ... [107 lines] ...

        - l1
        populations:
        - l1_exc
        model: my_multimeter
      connection_recorders:
      - source_layers:
        - l1
        source_population: l1_exc
        target_layers:
        - l1
        target_population: l1_inh
        connection_model: conn_1_AMPA
        model: weight_recorder
    nest_params: {}
  

#### We can save the parameter tree defining the whole network...

In [42]:
net.tree.write(PARAMS_DIR/'network_tree.yml')

PosixPath('params/network_tree.yml')

#### And use it to recreate another identical network

In [47]:
net2 = Network(ParamsTree.read(PARAMS_DIR/'network_tree.yml'))

2020-05-29 08:40:27,959 [denest.network] INFO: Build N=2 ``Model`` objects
2020-05-29 08:40:27,960 [denest.network] INFO: Build N=2 ``SynapseModel`` objects
2020-05-29 08:40:27,961 [denest.network] INFO: Build N=2 ``Model`` objects
2020-05-29 08:40:27,962 [denest.network] INFO: Build N=2 ``Layer`` or ``InputLayer`` objects.
2020-05-29 08:40:27,964 [denest.utils.validation] INFO: Object `conn_1_AMPA`: params: using default value for optional parameters:
{'type': 'topological'}
2020-05-29 08:40:27,966 [denest.utils.validation] INFO: Object `conn_2_GABAA`: params: using default value for optional parameters:
{'type': 'topological'}
2020-05-29 08:40:27,967 [denest.network] INFO: Build N=2 ``ConnectionModel`` objects
2020-05-29 08:40:27,970 [denest.network] INFO: Build N=3 ``TopoConnection`` objects
2020-05-29 08:40:27,974 [denest.network] INFO: Build N=1 population recorders.
2020-05-29 08:40:27,976 [denest.network] INFO: Build N=1 connection recorders.
2020-05-29 08:40:27,981 [denest.netw

# Change the network's state

# Let's familiarize ourselves with the ``Simulation`` object


## Initialize an empty Simulation object

Empty network, no kernel initialization, etc...

In [48]:
sim = Simulation(output_dir=OUTPUT_DIR)

2020-05-29 08:40:30,128 [denest.utils.validation] INFO: 'None' tree: adding empty child kernel
2020-05-29 08:40:30,130 [denest.utils.validation] INFO: 'None' tree: adding empty child simulation
2020-05-29 08:40:30,131 [denest.utils.validation] INFO: 'None' tree: adding empty child session_models
2020-05-29 08:40:30,132 [denest.utils.validation] INFO: 'None' tree: adding empty child network
2020-05-29 08:40:30,135 [denest.utils.validation] INFO: Object `simulation`: params: using default value for optional parameters:
{'input_dir': 'input', 'output_dir': 'output', 'sessions': []}
2020-05-29 08:40:30,137 [denest.utils.validation] INFO: Object `kernel`: params: using default value for optional parameters:
{'extension_modules': [], 'nest_seed': 1, 'python_seed': 1}
2020-05-29 08:40:30,139 [denest.simulation] INFO: Initializing NEST kernel and seeds...
2020-05-29 08:40:30,140 [denest.simulation] INFO:   Resetting NEST kernel...
2020-05-29 08:40:30,151 [denest.simulation] INFO:   Setting NES

In [49]:
# The data and metadata is saved there. For now it's all empty
!ls {OUTPUT_DIR}

data               git_hash           parameter_tree.yml session_times.yml


## Initialize the NEST kernel

``Simulation.init_kernel??`` for doc

In [50]:
kernel_tree = {
    'params':
        {
            'nest_seed': 10,
            'extension_modules': [],
        },
    'nest_params':
        {
            'resolution': 0.5,
        },
}

In [51]:
sim.init_kernel(kernel_tree)

2020-05-29 08:40:31,035 [denest.utils.validation] INFO: Object `kernel`: params: using default value for optional parameters:
{'python_seed': 1}
2020-05-29 08:40:31,039 [denest.simulation] INFO: Initializing NEST kernel and seeds...
2020-05-29 08:40:31,041 [denest.simulation] INFO:   Resetting NEST kernel...
2020-05-29 08:40:31,048 [denest.simulation] INFO:   Setting NEST kernel status...
2020-05-29 08:40:31,049 [denest.simulation] INFO:     Calling `nest.SetKernelStatus({'resolution': 0.5})`
2020-05-29 08:40:31,051 [denest.simulation] INFO:     Calling `nest.SetKernelStatus({'data_path': 'output/data', 'grng_seed': 11, 'rng_seeds': range(12, 13)})
2020-05-29 08:40:31,053 [denest.simulation] INFO:   Finished setting NEST kernel status
2020-05-29 08:40:31,055 [denest.simulation] INFO:   Installing external modules...
2020-05-29 08:40:31,056 [denest.simulation] INFO:   Finished installing external modules
2020-05-29 08:40:31,057 [denest.simulation] INFO:   Setting Python seed: 1
2020-05-

In [52]:
# nest_params have been passed to nest.SetKernelStatus
nest.GetKernelStatus('resolution')

0.5

In [53]:
# The raw data will be saved in the output directory
nest.GetKernelStatus('data_path')

'output/data'

In [54]:
# The kernel params are saved in the simulation's tree
sim.tree.children['kernel']

ParamsTree(name='kernel', parent='None')
  params:
    nest_seed: 10
    extension_modules: []
  nest_params:
    resolution: 0.5
  

## Create a network

We build and create the same network by passing the network tree,
using the ``Simulation.create_network`` method

In [55]:
net_tree = ParamsTree.read(PARAMS_DIR/'network_tree.yml')

In [56]:
sim.create_network(net_tree)

2020-05-29 08:40:33,350 [denest.simulation] INFO: Building network.
2020-05-29 08:40:33,381 [denest.network] INFO: Build N=2 ``Model`` objects
2020-05-29 08:40:33,382 [denest.network] INFO: Build N=2 ``SynapseModel`` objects
2020-05-29 08:40:33,386 [denest.network] INFO: Build N=2 ``Model`` objects
2020-05-29 08:40:33,388 [denest.network] INFO: Build N=2 ``Layer`` or ``InputLayer`` objects.
2020-05-29 08:40:33,389 [denest.utils.validation] INFO: Object `conn_1_AMPA`: params: using default value for optional parameters:
{'type': 'topological'}
2020-05-29 08:40:33,392 [denest.utils.validation] INFO: Object `conn_2_GABAA`: params: using default value for optional parameters:
{'type': 'topological'}
2020-05-29 08:40:33,393 [denest.network] INFO: Build N=2 ``ConnectionModel`` objects
2020-05-29 08:40:33,396 [denest.network] INFO: Build N=3 ``TopoConnection`` objects
2020-05-29 08:40:33,401 [denest.network] INFO: Build N=1 population recorders.
2020-05-29 08:40:33,403 [denest.network] INFO: 

In [57]:
# Network object was created and can be accessed as we learnt
sim.network.layers['l1'].gid

(52,)

In [58]:
# network tree was saved and can be re-used
sim.tree.children['network']

ParamsTree(name='network', parent='None')
  params: {}
  nest_params: {}
  neuron_models:
    params: {}
    nest_params: {}
    my_neuron:
      params:
        nest_model: ht_neuron
      nest_params:
        g_KL: 1.0
        g_NaL: 1.0
      l1_exc:
        params: {}
        nest_params:
          V_m: -44.0
  
  ... [107 lines] ...

        - l1
        populations:
        - l1_exc
        model: my_multimeter
      connection_recorders:
      - source_layers:
        - l1
        source_population: l1_exc
        target_layers:
        - l1
        target_population: l1_inh
        connection_model: conn_1_AMPA
        model: weight_recorder
    nest_params: {}
  

## Create some sessions

### so we can run the network in specific conditions

### Build session models from a tree

In [59]:
# Notice the hierarchical inheritance as before
session_models_tree = ParamsTree.read(PARAMS_DIR/'session_models.yml').children['session_models']
print(session_models_tree)

params:
  record: true
  simulation_time: 100.0
nest_params: {}
warmup:
  params:
    record: false
  nest_params: {}
high_input:
  params:
    unit_changes:
    - layers:
      - input_layer
      population: poisson_generator
      params:
        rate: 100.0
  nest_params: {}
low_input:
  params:
    unit_changes:
    - layers:
      - input_layer
      population: poisson_generator
      params:
        rate: 50.0
  nest_params: {}



In [60]:
sim.build_session_models(session_models_tree)

2020-05-29 08:40:35,702 [denest.simulation] INFO: Build N=3 session models


In [61]:
sim.session_models

{'warmup': ParamsTree(name='warmup', parent='session_models')
   params:
     record: false
   nest_params: {}
   ,
 'high_input': ParamsTree(name='high_input', parent='session_models')
   params:
     unit_changes:
     - layers:
       - input_layer
       population: poisson_generator
       params:
         rate: 100.0
   nest_params: {}
   ,
 'low_input': ParamsTree(name='low_input', parent='session_models')
   params:
     unit_changes:
     - layers:
       - input_layer
       population: poisson_generator
       params:
         rate: 50.0
   nest_params: {}
   }

### Build a list of sessions from the ``session_models`` templates

In [62]:
sessions_order = ['warmup', 'high_input', 'low_input', 'high_input', 'low_input']

In [63]:
sim.build_sessions(sessions_order)

2020-05-29 08:40:37,572 [denest.simulation] INFO: Build N=5 sessions
2020-05-29 08:40:37,580 [denest.session] INFO: Creating session "00_warmup"
2020-05-29 08:40:37,583 [denest.utils.validation] INFO: Object `00_warmup`: params: using default value for optional parameters:
{'inputs': {},
 'reset_network': False,
 'synapse_changes': [],
 'unit_changes': []}
2020-05-29 08:40:37,586 [denest.session] INFO: Creating session "01_high_input"
2020-05-29 08:40:37,590 [denest.utils.validation] INFO: Object `01_high_input`: params: using default value for optional parameters:
{'inputs': {}, 'reset_network': False, 'synapse_changes': []}
2020-05-29 08:40:37,592 [denest.session] INFO: Creating session "02_low_input"
2020-05-29 08:40:37,595 [denest.utils.validation] INFO: Object `02_low_input`: params: using default value for optional parameters:
{'inputs': {}, 'reset_network': False, 'synapse_changes': []}
2020-05-29 08:40:37,600 [denest.session] INFO: Creating session "03_high_input"
2020-05-29 08

In [64]:
# Notice the session names
sim.sessions

[Session(00_warmup, {'inputs': {},
  'record': False,
  'reset_network': False,
  'simulation_time': 100.0,
  'synapse_changes': [],
  'unit_changes': []}),
 Session(01_high_input, {'inputs': {},
  'record': True,
  'reset_network': False,
  'simulation_time': 100.0,
  'synapse_changes': [],
  'unit_changes': [{'layers': ['input_layer'],
                    'params': {'rate': 100.0},
                    'population': 'poisson_generator'}]}),
 Session(02_low_input, {'inputs': {},
  'record': True,
  'reset_network': False,
  'simulation_time': 100.0,
  'synapse_changes': [],
  'unit_changes': [{'layers': ['input_layer'],
                    'params': {'rate': 50.0},
                    'population': 'poisson_generator'}]}),
 Session(03_high_input, {'inputs': {},
  'record': True,
  'reset_network': False,
  'simulation_time': 100.0,
  'synapse_changes': [],
  'unit_changes': [{'layers': ['input_layer'],
                    'params': {'rate': 100.0},
                    'population': 'po

### Run all the sessions

In [65]:
print('kernel time: ', nest.GetKernelStatus('time'))

kernel time:  0.0


In [66]:
sim.run()

2020-05-29 08:40:38,811 [denest.simulation] INFO: Running 5 sessions...
2020-05-29 08:40:38,811 [denest.simulation] INFO: Running session: '00_warmup'...
2020-05-29 08:40:38,813 [denest.session] INFO: Initializing session...
-> Changing synapses's state.: 0it [00:00, ?it/s]
2020-05-29 08:40:38,821 [denest.network.recorders] INFO:   Setting status for recorder my_multimeter_l1_l1_exc: {'start': 100.0}
2020-05-29 08:40:38,822 [denest.network.recorders] INFO:   Setting status for recorder weight_recorder_conn_1_AMPA-l1-l1_exc-l1-l1_inh: {'start': 100.0}
2020-05-29 08:40:38,824 [denest.session] INFO: Finished initializing session

2020-05-29 08:40:38,825 [denest.session] INFO: Running session '00_warmup' for 100 ms
2020-05-29 08:40:38,954 [denest.session] INFO: Finished running session
2020-05-29 08:40:38,956 [denest.session] INFO: Session '00_warmup' virtual running time: 100 ms
2020-05-29 08:40:38,960 [denest.session] INFO: Session '00_warmup' real running time: 0h:00m:00s
2020-05-29 08:

In [67]:
print('kernel time: ', nest.GetKernelStatus('time'))

kernel time:  500.0


### Save the simulation metadata

In [68]:
sim.save_metadata()

2020-05-29 08:40:40,529 [denest.simulation] INFO: Saving simulation metadata...
2020-05-29 08:40:40,542 [denest.simulation] INFO: Creating output directory: output
2020-05-29 08:40:40,546 [denest.io.save] INFO: Clearing directory: output
2020-05-29 08:40:40,549 [denest.io.save] INFO: Clearing directory: output
2020-05-29 08:40:40,552 [denest.io.save] INFO: Clearing directory: output/data
2020-05-29 08:40:40,557 [denest.io.save] INFO: Clearing directory: output/data
2020-05-29 08:40:40,561 [denest.io.save] INFO: Clearing directory: output/data
2020-05-29 08:40:40,565 [denest.io.save] INFO: Clearing directory: output
2020-05-29 08:40:40,758 [denest.simulation] INFO: Finished saving simulation metadata


In [69]:
!ls {OUTPUT_DIR}

data               git_hash           parameter_tree.yml session_times.yml


In [70]:
!cat {OUTPUT_DIR/'session_times.yml'}

00_warmup: !!python/tuple
- 0.0
- 100.0
01_high_input: !!python/tuple
- 100.0
- 200.0
02_low_input: !!python/tuple
- 200.0
- 300.0
03_high_input: !!python/tuple
- 300.0
- 400.0
04_low_input: !!python/tuple
- 400.0
- 500.0


In [71]:
!ls {OUTPUT_DIR/'data'}

my_multimeter_l1_l1_exc.yml
weight_recorder_conn_1_AMPA-l1-l1_exc-l1-l1_inh.yml


## Load simulation outputs

In [ ]:
import denest.io.load
from pathlib import Path

OUTPUT_DIR = Path(sim.output_dir)

In [ ]:
# Load the start and end time for each session
session_times = denest.io.load.load_session_times(OUTPUT_DIR) 
print(session_times)  # {<session_name>: (<session_start>, <session_end>)}

In [ ]:
## Load data from a specific recorder.

recorder_metadata_path = OUTPUT_DIR/'data/multimeter_l1_l1_exc.yml'

# All relevant information about this recorder and the population it's
# connected to are contained in its metadata file
recorder_metadata = denest.io.load.load_yaml(recorder_metadata_path)
print(f'Metadata keys: {recorder_metadata.keys()}')

In [ ]:
# We can load the raw data as pandas dataframe
df = denest.io.load.load(recorder_metadata_path)
print(df[0:5])

In [ ]:
all_recorder_metadata_paths = denest.io.load.metadata_paths(OUTPUT_DIR)
print(all_recorder_metadata_paths)

In [ ]:
for metadata_path in all_recorder_metadata_paths:
  print(f'Recorder: {metadata_path.name}')
  print(f'{denest.io.load.load(metadata_path)[0:5]}\n')

## Replicate the simulation

In [ ]:
params = ParamsTree.read(OUTPUT_DIR/'parameter_tree.yml')

In [ ]:
# sim = denest.Simulation(params)
# sim.run()